In [1]:
import numpy as np
import pandas as pd

In [2]:
df = pd.read_csv("maindata.csv")#,header=1, engine='openpyxl')

In [3]:
df_for_classifier = df[['Gender', 'Ht', 'Weight', 'BMI', 'HoarseVoice',
       'DiffSwal', 'PainSwal', 'Otalgia', 'UnilatNO', 'NeckLump', 'ThroatLump',
       'FOSIT', 'SoreThroat',"Diagnostic group"]]
df_for_classifier.head()

,Gender,Ht,Weight,BMI,HoarseVoice,DiffSwal,PainSwal,Otalgia,UnilatNO,NeckLump,ThroatLump,FOSIT,SoreThroat,Diagnostic group
0,M,1.41,28.95,14.561642,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,Non-cancer
1,M,1.67,52.50,18.824626,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,Non-cancer
2,M,1.78,61.00,19.094811,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,Non-cancer
3,M,1.87,67.50,19.302811,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,Non-cancer
4,M,1.70,56.80,19.653979,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Non-cancer


In [4]:
df_for_classifier['Gender'] = df_for_classifier['Gender'].map({'M': 1, 'F': 0})
df_for_classifier['Diagnostic group'] = df_for_classifier['Diagnostic group'].map({'Cancer': 1, 'Non-cancer': 0})
df_for_classifier.head()

/tmp/ipykernel_4667/3131253531.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_for_classifier['Gender'] = df_for_classifier['Gender'].map({'M': 1, 'F': 0})
/tmp/ipykernel_4667/3131253531.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_for_classifier['Diagnostic group'] = df_for_classifier['Diagnostic group'].map({'Cancer': 1, 'Non-cancer': 0})


,Gender,Ht,Weight,BMI,HoarseVoice,DiffSwal,PainSwal,Otalgia,UnilatNO,NeckLump,ThroatLump,FOSIT,SoreThroat,Diagnostic group
0,1,1.41,28.95,14.561642,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1,1,1.67,52.50,18.824626,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2,1,1.78,61.00,19.094811,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,1,1.87,67.50,19.302811,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,1,1.70,56.80,19.653979,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [6]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler
scaler = MinMaxScaler()
#df_for_classifier['Age'] = scaler.fit_transform(df_for_classifier[['Age']])
df_for_classifier['Ht'] = scaler.fit_transform(df_for_classifier[['Ht']])
df_for_classifier['Weight'] = scaler.fit_transform(df_for_classifier[['Weight']])
df_for_classifier.head()


/tmp/ipykernel_4667/3440211753.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_for_classifier['Ht'] = scaler.fit_transform(df_for_classifier[['Ht']])
/tmp/ipykernel_4667/3440211753.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_for_classifier['Weight'] = scaler.fit_transform(df_for_classifier[['Weight']])


,Gender,Ht,Weight,BMI,HoarseVoice,DiffSwal,PainSwal,Otalgia,UnilatNO,NeckLump,ThroatLump,FOSIT,SoreThroat,Diagnostic group
0,1,0.035088,0.000000,14.561642,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1,1,0.491228,0.029960,18.824626,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2,1,0.684211,0.040773,19.094811,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,1,0.842105,0.049043,19.302811,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,1,0.543860,0.035430,19.653979,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [7]:
df_for_classifier.columns

Index(['Gender', 'Ht', 'Weight', 'BMI', 'HoarseVoice', 'DiffSwal', 'PainSwal',
       'Otalgia', 'UnilatNO', 'NeckLump', 'ThroatLump', 'FOSIT', 'SoreThroat',
       'Diagnostic group'],
      dtype='object')

In [13]:
from sklearn.model_selection import StratifiedKFold, cross_val_predict
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.linear_model import LogisticRegression
pd.options.mode.chained_assignment = None
# Fill NaNs with 0
df_for_classifier.fillna(0, inplace=True)

# Define features and target variable
X = df_for_classifier[[ 'Gender', 'Ht', 'Weight', 'BMI', 'HoarseVoice', 'DiffSwal',
       'PainSwal', 'Otalgia', 'UnilatNO', 'NeckLump', 'ThroatLump', 'FOSIT',
       'SoreThroat']]
y = df_for_classifier['Diagnostic group']

# Initialize StratifiedKFold
log_reg = LogisticRegression(class_weight={0:1,1:10000}, max_iter=10000)
skf = StratifiedKFold(n_splits=10)

# List to store confusion matrices for each fold
conf_matrices = []


conf_matrices = []

for train_index, test_index in skf.split(X, y):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    
    # Train the model
    log_reg.fit(X_train, y_train)
    
    # Make predictions
    y_pred = log_reg.predict(X_test)
    
    # Compute the confusion matrix for this fold
    conf_matrix = confusion_matrix(y_test, y_pred)
    
    # Normalize the confusion matrix so that all elements sum to 100%
    conf_matrix_normalized = conf_matrix / conf_matrix.sum() * 100
    conf_matrices.append(conf_matrix_normalized)

# Convert list of confusion matrices to a numpy array
conf_matrices = np.array(conf_matrices)

# Calculate the mean confusion matrix
mean_conf_matrix = np.mean(conf_matrices, axis=0)

# Calculate the standard deviation of the confusion matrices
std_dev_conf_matrix = np.std(conf_matrices, axis=0)

# Round the mean and standard deviation confusion matrices to 1 decimal place
mean_conf_matrix_rounded = np.round(mean_conf_matrix, 1)
std_dev_conf_matrix_rounded = np.round(std_dev_conf_matrix, 1)

# Print the rounded mean confusion matrix
print("Mean Confusion Matrix (Percentages, Rounded to 1 dp):")
print(mean_conf_matrix_rounded)

# Print the rounded standard deviation of the confusion matrices
print("Standard Deviation of Confusion Matrices (Percentages, Rounded to 1 dp):")
print(std_dev_conf_matrix_rounded)

# Optionally, print the classification report for the overall predictions
y_pred = cross_val_predict(log_reg, X, y, cv=10)
class_report = classification_report(y, y_pred)
print("Classification Report:")
print(class_report)

Mean Confusion Matrix (Percentages, Rounded to 1 dp):
[[26.2 69.3]
 [ 0.7  3.8]]
Standard Deviation of Confusion Matrices (Percentages, Rounded to 1 dp):
[[11.5 11.5]
 [ 1.   1. ]]
Classification Report:
              precision    recall  f1-score   support

         0.0       0.97      0.27      0.43       424
         1.0       0.05      0.85      0.10        20

    accuracy                           0.30       444
   macro avg       0.51      0.56      0.26       444
weighted avg       0.93      0.30      0.41       444

